In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('Data.csv')

In [2]:
df.head()

,Date_Time,SO2 (ppb),NOx (ppb),CO (ppm),CO (8hr),O3,O3 (8hr),PM2.5 (ug/m3),PM10 (ug/m3)
0,11/1/2012 01:00,5.05,22.12,0.88,NaN,0.18,NaN,68.32,84.27
1,11/1/2012 02:00,6.23,26.84,0.86,NaN,0.21,NaN,79.02,100.74
2,11/1/2012 03:00,6.30,31.43,0.83,NaN,0.19,NaN,110.64,133.44
3,11/1/2012 04:00,6.32,37.55,0.81,NaN,0.23,NaN,97.79,125.52
4,11/1/2012 05:00,6.00,36.56,0.79,NaN,0.12,NaN,120.86,142.32


In [3]:
df['Date_Time'] = df['Date_Time'].str.replace(' 24:00', ' 00:00')
df['Date_Time'] = pd.to_datetime(df['Date_Time'], dayfirst=True, errors='coerce')

In [4]:
daily_avg = df.groupby(df['Date_Time'].dt.date).mean()
daily_avg.head()

,Date_Time,SO2 (ppb),NOx (ppb),CO (ppm),CO (8hr),O3,O3 (8hr),PM2.5 (ug/m3),PM10 (ug/m3)
Date_Time,,,,,,,,,
2012-01-11,2012-01-11 11:30:00,5.544583,22.977917,0.811250,0.722132,0.723333,0.890809,91.745652,154.165652
2012-01-12,2012-01-12 11:30:00,5.935000,38.001000,1.061000,1.256077,3.974000,2.462251,206.534000,235.335000
2012-02-11,2012-02-11 11:30:00,5.185833,15.049167,0.781667,0.848750,0.891818,1.366719,83.719583,150.599167
2012-02-12,2012-02-12 11:30:00,6.681875,41.185385,1.304545,0.820804,4.766923,6.233619,214.111667,242.520625
2012-03-11,2012-03-11 11:30:00,5.101667,13.022500,0.646250,0.660885,0.200000,0.246615,62.883913,102.994167


In [5]:
daily_avg['CO (8hr)'] = df.groupby(df['Date_Time'].dt.date).max()['CO (8hr)'].values
daily_avg['O3 (8hr)'] = df.groupby(df['Date_Time'].dt.date).max()['O3 (8hr)'].values
daily_avg.head()

,Date_Time,SO2 (ppb),NOx (ppb),CO (ppm),CO (8hr),O3,O3 (8hr),PM2.5 (ug/m3),PM10 (ug/m3)
Date_Time,,,,,,,,,
2012-01-11,2012-01-11 11:30:00,5.544583,22.977917,0.811250,0.97500,0.723333,1.568750,91.745652,154.165652
2012-01-12,2012-01-12 11:30:00,5.935000,38.001000,1.061000,2.06000,3.974000,7.257500,206.534000,235.335000
2012-02-11,2012-02-11 11:30:00,5.185833,15.049167,0.781667,1.28875,0.891818,3.660000,83.719583,150.599167
2012-02-12,2012-02-12 11:30:00,6.681875,41.185385,1.304545,1.54125,4.766923,7.192857,214.111667,242.520625
2012-03-11,2012-03-11 11:30:00,5.101667,13.022500,0.646250,0.81625,0.200000,0.711250,62.883913,102.994167


In [6]:
daily_avg.set_index('Date_Time', inplace=True)
daily_avg = daily_avg[daily_avg.index.year != 2012]

In [7]:
## PM2.5 Sub-Index calculation
def get_PM25_subindex(x):
    if x <= 30:
        return x * 50 / 30
    elif x <= 60:
        return 50 + (x - 30) * 50 / 30
    elif x <= 90:
        return 100 + (x - 60) * 100 / 30
    elif x <= 120:
        return 200 + (x - 90) * 100 / 30
    elif x <= 250:
        return 300 + (x - 120) * 100 / 130
    elif x > 250:
        return 400 + (x - 250) * 100 / 130
    else:
        return 0

daily_avg["PM2.5_subindex"] = daily_avg["PM2.5 (ug/m3)"].apply(lambda x: get_PM25_subindex(x))

In [8]:
## PM10 Sub-Index calculation
def get_PM10_subindex(x):
    if x <= 50:
        return x
    elif x <= 100:
        return x
    elif x <= 250:
        return 100 + (x - 100) * 100 / 150
    elif x <= 350:
        return 200 + (x - 250)
    elif x <= 430:
        return 300 + (x - 350) * 100 / 80
    elif x > 430:
        return 400 + (x - 430) * 100 / 80
    else:
        return 0

daily_avg["PM10_subindex"] = daily_avg["PM10 (ug/m3)"].apply(lambda x: get_PM10_subindex(x))

In [9]:
## SO2 Sub-Index calculation
def get_SO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 380:
        return 100 + (x - 80) * 100 / 300
    elif x <= 800:
        return 200 + (x - 380) * 100 / 420
    elif x <= 1600:
        return 300 + (x - 800) * 100 / 800
    elif x > 1600:
        return 400 + (x - 1600) * 100 / 800
    else:
        return 0

daily_avg["SO2_subindex"] = daily_avg["SO2 (ppb)"].apply(lambda x: get_SO2_subindex(x))

In [10]:
## NOx Sub-Index calculation
def get_NOx_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 180:
        return 100 + (x - 80) * 100 / 100
    elif x <= 280:
        return 200 + (x - 180) * 100 / 100
    elif x <= 400:
        return 300 + (x - 280) * 100 / 120
    elif x > 400:
        return 400 + (x - 400) * 100 / 120
    else:
        return 0

daily_avg["NOx_subindex"] = daily_avg["NOx (ppb)"].apply(lambda x: get_NOx_subindex(x))

In [11]:
## CO Sub-Index calculation
def get_CO_subindex(x):
    if x <= 1:
        return x * 50 / 1
    elif x <= 2:
        return 50 + (x - 1) * 50 / 1
    elif x <= 10:
        return 100 + (x - 2) * 100 / 8
    elif x <= 17:
        return 200 + (x - 10) * 100 / 7
    elif x <= 34:
        return 300 + (x - 17) * 100 / 17
    elif x > 34:
        return 400 + (x - 34) * 100 / 17
    else:
        return 0

daily_avg["CO_subindex"] = daily_avg["CO (8hr)"].apply(lambda x: get_CO_subindex(x))


In [12]:
## O3 Sub-Index calculation
def get_O3_subindex(x):
    if x <= 50:
        return x * 50 / 50
    elif x <= 100:
        return 50 + (x - 50) * 50 / 50
    elif x <= 168:
        return 100 + (x - 100) * 100 / 68
    elif x <= 208:
        return 200 + (x - 168) * 100 / 40
    elif x <= 748:
        return 300 + (x - 208) * 100 / 539
    elif x > 748:
        return 400 + (x - 400) * 100 / 539
    else:
        return 0

daily_avg["O3_subindex"] = daily_avg["O3 (8hr)"].apply(lambda x: get_O3_subindex(x))


In [13]:
## AQI bucketing
def get_AQI_bucket(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Satisfactory"
    elif x <= 200:
        return "Moderate"
    elif x <= 300:
        return "Poor"
    elif x <= 400:
        return "Very Poor"
    elif x > 400:
        return "Severe"
    else:
        return np.NaN

daily_avg["Checks"] = (daily_avg["PM2.5_subindex"] > 0).astype(int) + \
                (daily_avg["PM10_subindex"] > 0).astype(int) + \
                (daily_avg["SO2_subindex"] > 0).astype(int) + \
                (daily_avg["NOx_subindex"] > 0).astype(int) + \
                (daily_avg["CO_subindex"] > 0).astype(int) + \
                (daily_avg["O3_subindex"] > 0).astype(int)

In [14]:
daily_avg["AQI_calculated"] = round(daily_avg[["PM2.5_subindex", "PM10_subindex", "SO2_subindex", "NOx_subindex",
                                 "CO_subindex", "O3_subindex"]].max(axis = 1))

daily_avg.loc[daily_avg["PM2.5_subindex"] + daily_avg["PM10_subindex"] <= 0, "AQI_calculated"] = np.NaN
daily_avg.loc[daily_avg.Checks < 3, "AQI_calculated"] = np.NaN

daily_avg["AQI_bucket_calculated"] = daily_avg["AQI_calculated"].apply(lambda x: get_AQI_bucket(x))

In [16]:
daily_avg.head()

,SO2 (ppb),NOx (ppb),CO (ppm),CO (8hr),O3,O3 (8hr),PM2.5 (ug/m3),PM10 (ug/m3),PM2.5_subindex,PM10_subindex,SO2_subindex,NOx_subindex,CO_subindex,O3_subindex,Checks,AQI_calculated,AQI_bucket_calculated
Date_Time,,,,,,,,,,,,,,,,,
2013-01-01 11:30:00,12.557917,78.851304,2.261000,2.60125,4.500000,7.19625,258.470476,372.176875,406.515751,327.721094,15.697396,98.564130,107.515625,7.19625,6,407.0,Severe
2013-01-02 11:30:00,49.263750,90.861667,2.052083,2.37250,5.193333,9.68875,309.099583,440.492917,445.461218,413.116146,61.579687,110.861667,104.656250,9.68875,6,445.0,Severe
2013-01-03 11:30:00,27.795417,41.032632,0.802500,1.27500,16.061250,37.32500,141.596087,294.745833,316.612375,244.745833,34.744271,51.290789,63.750000,37.32500,6,317.0,Very Poor
2013-01-04 11:30:00,6.973913,12.170870,0.170588,0.26800,5.864545,9.43000,50.000000,111.009000,83.333333,107.339333,8.717391,15.213587,13.400000,9.43000,6,107.0,Moderate
2013-01-05 11:30:00,11.516667,18.224444,0.018333,0.05000,8.751667,14.66875,31.025556,91.124118,51.709259,91.124118,14.395833,22.780556,2.500000,14.66875,6,91.0,Satisfactory


In [17]:
daily_avg.to_csv('daily_avg.csv', index=True)